In [ ]:
#Importamos librerías importantes a usar
import numpy as np
from scipy.integrate import solve_ivp
from scipy.integrate import solve_bvp
from scipy import constants as ctes

import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline #para que se vean bien los gráficos en jupyter notebook:

UsageError: unrecognized arguments: #para que se vean bien los graficos en juupyter notebook:


# Modelo y análisis de Leine et al. 2009

In [ ]:
#Definimos el modelo como un sistema de odes de 1 orden.
def leine(t,cond_init,):



    return tuple

#Definimos un intervalo de 40 segundos
time = np.linspace(0,40,100)

#seteamos los argumentos del modelo


leine_args= []

#seteamos las condiciones iniciales


cond_init = []

#Resolvemos el sistema
sol_leine = solve_ivp(leine, t_span=time,y0=cond_init, args= )

# Modelo de Cendra y Díaz et al. 2007

In [ ]:
def cendra():

    return tuple

time = np.linspace(0,40,100) #Definimos un intervalo de 40 segundos
